<h1>Convolutional Neural Networks anhand von MNIST</h1>

<p>Das neuronale Netzwerk der letzten &Uuml;bung brauchte nur einige hundert Megabyte Arbeitsspeicher. Die 60000 MNIST Bilder mit ihren 784 Pixeln, die Gewichtsmatrizen und alle Zwischenergebnisse, die im Netzwerk beim Vorw&auml;rtspass berechnet wurden, sind keine 350MB gro&szlig;:<br />
(60000&lowast; 784 + 784&lowast; 300 + 60000 &lowast; 300 + 300 &lowast; 10 + 60000 &lowast; 10) &lowast; 4 = 335MB <br />(MNISTData + Weights1 + Intermediate + Weight2 + Predictions) &lowast; Float32</p>

<p>Dieser Verbrauch steigt rasant an, wenn Konvolutionsfilter ins Spiel kommen. Werden die Eingangsdaten mit 10 Filtern beliebiger Gr&ouml;&szlig;e (z.b. 3x3) gefalten, sind die ausgehenden Daten 10 mal so gro&szlig;:<br />
(60000&lowast;784+10&lowast;3&lowast;3+60000&lowast;10&lowast;784)&lowast;4=2GB</p>

<p>Um das zu verhindern, sollten in Zukunft nicht mehr alle Daten auf einmal im Netzwerk verarbeitet werden. Stattdessen werden Mini-Batches ben&ouml;tigt.</p>

<p>Das komplette Notebook steht wieder zum <a href="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung4/Tensorflow_ConvNet_MNIST_Vorlage.ipynb">download</a> bereit.</p>

<hr>

<h2>Neural Networks mit Mini-Batches</h2>

<p>Im folgenden ist ein zweischichtiges neuronales Netzwerk implementiert, welches alle MNIST Ziffern auf einmal verarbeitet. Bauen Sie den Code so um, dass er stattdessen mit Mini-Batches funktioniert. Dabei&nbsp;k&ouml;nnen Sie Numpy verwenden und die Daten als Mini-Batch in den Computation-Graph von Tensorflow geben&nbsp;oder Sie benutzen Tensorflows Batch-Methoden, um die Batches innerhalb eines Graphens zu erzeugen. Die K&ouml;nigsdiziplin sind Tensorflow Esitmators, die die Arbeit des Batchings &uuml;bernehmen, aber viele andere Anforderungen an das Netzwerk haben.&nbsp;Wichtig ist in allen&nbsp;F&auml;llen, dass auch die Testdaten gebatched werden.</p>

<ul>
	<li><strong>Numpy</strong>: Es ist m&ouml;glich die Daten einmalig in kleine Batches zu unterteilen und diese dann in zuf&auml;lliger Reihnfolge in den Computation-Graph zu geben. Besser jedoch ist die Variante, bei der erst im letzten Moment ein Batch aus dem gesamten Datensatz extrahiert wird. Der Extraktionsbereich sollte dabei zuf&auml;llig gew&auml;hlt sein.&nbsp;&nbsp;</li>
	<li><strong>Tensorflow Batch</strong>: Sind die&nbsp;Daten klein genug, dass Sie&nbsp;in den Arbeitsspeicher, aber nicht durch das Netzwerk passen. K&ouml;nnen Sie zun&auml;chst&nbsp;komplett in den Graphen geladen werden und von dort in kleine Batches, mit einem <a href="http://www.tensorflow.org/api_docs/python/tf/train/slice_input_producer">Slice Producer</a>, zerlegt werden. Der Slicer braucht einen <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/train/Coordinator">Coordinator</a>, der das Zerlegen der Daten mithilfe von mehreren Threads koordiniert.</li>
	<li><strong>Tensorflow Estimator</strong>: Innerhalb der High-Level API von Tensorflow gibt es die Möglichkeit, Estimators zu verwenden. Diese übernehmen sämtliche Batching-Arbeiten, verlangen aber bestimmte Eigenschaften vom Computation-Graphen. So m&uuml;ssen Trainings- und Evaluierungsmethoden in einen sogenannten <a href="https://www.tensorflow.org/api_docs/python/tf/estimator/EstimatorSpec">EstimatorSpec</a> beschrieben werden, um sp&auml;ter mit einen <a href="http://www.tensorflow.org/api_docs/python/tf/estimator">Estimator Model</a> arbeiten zu k&ouml;nnen.</li>
</ul>

<p>Der Programmcode für den Import der MNIST Zahlen wurde in die <a href="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung4/cvutils.py">cvutils.py</a> Datei ausgelagert. Bitte kopieren Sie die Datei in den Ordner, in dem das Notebook läuft. Ansonsten wird der Import "from cvutils import fetch_mnist" im nächsten Abschnitt fehlschlagen.</p> 

In [29]:
import tensorflow as tf
import math
import numpy as np

In [30]:
from cvutils import fetch_mnist
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state

# input and output data
mnist = fetch_mnist()
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

# shuffle data
random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]

# split data, both sizes are a multiple of 2048
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=59392, test_size=10240)

# normalize
X_train = (X_train / 255.) 
X_test = (X_test / 255.)

In [31]:
# pixel count
num_input = 28 * 28

# num of classes
num_classes = 10

# learn rate
learning_rate = 0.005

batch_size = 32
num_iters = math.ceil(X_train.shape[0] / batch_size)

# computation graph
graph = tf.Graph()
with graph.as_default():
    
    # input data with fix shape to infer shapes of other graph nodes a build time
    x_input = tf.placeholder(dtype=tf.float32, shape=[None, num_input], name='x')
    y_input = tf.placeholder(tf.int64, shape=[None], name='y')
        
    # two layer network: 784 -> 300 -> 10
    layer1 = tf.layers.dense(inputs=x_input, units=300, activation=tf.nn.relu)
    prediction = tf.layers.dense(inputs=layer1, units=num_classes)
    
    # compute trainings error
    cost = tf.losses.sparse_softmax_cross_entropy(labels=y_input, logits=prediction)
    
    # use the Adam optimizer to derive the cost function and update the weights
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # accuracy for multiple batches
    acc, update_acc = tf.metrics.accuracy(labels=y_input, predictions=tf.argmax(prediction, axis=-1))

In [37]:
# session configuration
config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True

# work on GPU if available
with tf.device("/gpu:0"):

    # start a new session
    with tf.Session(graph=graph, config=config) as session:  
    
        # initialize weights and bias variables
        session.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))     
        
        # which nodes to fetch from the computation graph
        fetch_train_nodes = {
            'cost' : cost,
            'optimizer' : optimizer 
        }
        
        for epoch in range(3):
            
            # shuffle data
            permutation = np.random.permutation(X_train.shape[0])
            X_train = X_train[permutation]
            y_train = y_train[permutation]
            
            for i in range(num_iters):
                X_batch = X_train[i * batch_size:(i + 1) * batch_size]
                y_batch = y_train[i * batch_size:(i + 1) * batch_size]
                
                output_batch = session.run(fetch_train_nodes, feed_dict={x_input: X_batch, y_input: y_batch})
                if(i % 500) == 0:
                    print("Train error [{}/{}]".format(epoch, i), output_batch["cost"])
            
        # check against test set
        print("Test accuracy ", session.run(update_acc, feed_dict={x_input: X_test, y_input: y_test}))

Train error [0/0] 2.27687
Train error [0/500] 0.0837313
Train error [0/1000] 0.0361951
Train error [0/1500] 0.00704363
Train error [1/0] 0.257601
Train error [1/500] 0.0187457
Train error [1/1000] 0.0129295
Train error [1/1500] 0.306597
Train error [2/0] 0.0233605
Train error [2/500] 0.0351897
Train error [2/1000] 0.196572
Train error [2/1500] 0.0649227
Test accuracy  0.967188


<hr>

<h2>Convolutional Neural Network mit MNIST Ziffern</h2>

<p>Nachdem das neuronale Netzwerk mit Mini-Batches arbeitet, k&ouml;nnen die Fully-Connected (Dense) Layer mit Konvolutionsschichten ersetzt werden. Sinnvoll sind z.B. zwei Schichten mit 64 5x5 und 96 3x3 Filterkerneln. Um die Dimensionalit&auml;t der Daten langsam zu reduzieren, k&ouml;nnen entweder Schrittweiten bei den Konvolutionsschichten eingestellt werden oder Pooling angewendet werden. Zum Schluss ist es hilfreich, die hochdimensionalen Daten zu flatten, um sie in Dense Layern auf 10 Dimensionen herunterzubrechen. Berechnen Sie wieder den Trainingsfehler und die Testgenauigkeit. Zu erwarten sind Genauigkeiten von bis zu 99%.&nbsp;</p>

<p>Je nachdem welche Tensorflow Version Sie nutzen (mindestens aber Version &gt;= 1.0), sind folgende Methoden hilfreich:</p>

<ul>
	<li><a href="https://www.tensorflow.org/api_docs/python/tf/nn/max_pool" target="_blank">tf.nn.max_pool</a> oder <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/max_pooling2d" target="_blank">tf.layers.max_pooling2d</a></li>
	<li><a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/conv2d" target="_blank">tf.nn.conv2d</a> oder <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/conv2d" target="_blank">tf.layers.conv2d</a></li>
	<li><a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/reshape" target="_blank">tf.reshape</a> oder <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/contrib/layers/flatten" target="_blank">tf.contrib.layers.flatten</a> oder <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/flatten" target="_blank">tf.layers.Flatten</a></li>
</ul>

<p><strong>Optional</strong>: Yann LeCun hat vor fast 20 Jahren das MNIST Datenset herausgebracht und die Convolutionsnetzwerke erfunden. Damals gab es nicht die nötige Rechenleistung um in kurzer Zeit die notwendigen Filterkernel mittels Backpropagation und Gradient Descent zu erlernen. Seine Netzwerke sind daher sehr minimalistisch. Implementieren Sie das <a href="http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf" target="_blank">LeNet5</a> Netzwerk nach seinen Vorbild. Padden Sie dazu die&nbsp;Eingangsdaten, damit die Bilder 32x32 Pixel haben und verwenden Sie nur 6, 16 und 120 Filterkernel&nbsp;(je 5x5 Pixel gro&szlig;) f&uuml;r die drei Konvolutionsschichten in LeNet5. Natürlich können Sie auch LeCun's <a href="http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf" target="_blank">Stochastic gradient descent</a> nutzen um ihr Netzwerk zu trainieren oder gar den <a href="https://arxiv.org/pdf/1412.6980v8.pdf" target="_blank">Adam Optimizer</a>.</p>

<p>&nbsp;</p>

<p><img alt="LeNet5" src="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung4/LeNet5.png" style="max-width: 790px; max-height: 100%; width: auto" /><br />
&nbsp;</p>

In [ ]:
# TODO Copy the neural network code and implement a convolutional version. Try to reach an accuracy over 96%.

<hr />

<h2>Abgabe</h2>

<p>Das von Ihnen erstellte Notebook muss sp&auml;testens bis zum 14. Januar 2018 um 23:59 UTC+1 ;) per E-Mail an&nbsp;<a href="mailto:hezel@htw-berlin.de" target="_blank">hezel@htw-berlin.de</a>&nbsp;eingesendet werden. Verwenden Sie als Betreff bitte &quot;CV1718 &Uuml;bung4 &lt;NAME&gt;&quot; und als Notebook Name &quot;CV1718_Ue4_Tensorflow_ConvNet_MNIST_NAME.ipynb&quot;. Bevor Sie mir eine Mail schicken, entfernen Sie bitte &uuml;ber &quot;Kernel&quot; -&gt; &quot;Restart and Clear Output&quot; s&auml;mtlichen von Python erstellten Inhalt und speichern anschlie&szlig;end das Notebook &quot;File&quot; -&gt; &quot;Save and Checkpoint&quot;.</p>